# NC-Global monthly temperature anomaly
This notebook provides an example on how to decode NC data using the netCDF4 library. In this example, we will read anomaly data from the CMA and display the result on a map.<br>
<br>
This data is a global monthly temperature anomaly.<br>
The average temperature used for the calculation is the average of the temperatures for the corresponding months over the 30-year period from 1961 to 1990.<br>
<br>
The data product will be shared on <https://data.cma.cn/en> from the end of September 2025, and it will also be shared via WIS2 in the near future.<br><br>
As part of the WIS2 Training you will be asked to download some data from the WIS2-broker hosted at wis2training-broker.wis2dev.io and use it in this notebook. If you want to run this notebook on you local machine, you can use the sample-data included in this notebook instead.<br>
<br>
Execute each block of code by selecting it and pressing `Shift+Enter` or clicking the `Run` button in the toolbar above.

# Data decoding

In [ ]:
import netCDF4
from netCDF4 import Dataset
import xarray as xr 

In [ ]:
file_path = "/root/sample-data/SURF_CLI_GLB_MST_MON_GRID_2DEG_202507.nc"

In [ ]:
dataset = xr.open_dataset(file_path)
dataset

# Visualization

Importing the required modules.<br>  
The following modules are imported:<br>
- netCDF4: Python interface to the NetCDF data format, enabling efficient reading/writing of NetCDF files (e.g., `Dataset` for file operations).  
- xarray: Library for labeled, multi-dimensional array data, simplifying operations on NetCDF-like datasets (e.g., `xr.Dataset` for intuitive data manipulation).  
- numpy: Numerical Python library for high-performance array computations and numerical data processing (e.g., `np.array` for array operations).  
- matplotlib: Plotting library for creating static, interactive, or animated visualizations (e.g., `plt` for figure and axis management).  
- cartopy: Cartographic library for geospatial data visualization, providing map projections and geographic features (e.g., `ccrs` for coordinate systems, `cfeature` for map elements like coastlines).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from netCDF4 import Dataset

In [ ]:
ncfile = Dataset(file_path)
lon = ncfile["lon"][:]
lat = ncfile["lat"][:]
ano = ncfile["anomaly"][:]
mon = ncfile["month"][:]

In [ ]:
# Extract month value (directly use .data attribute to get scalar value)
month_value = mon.data if hasattr(mon, "data") else mon

# Check latitude direction (reverse if data is South->North)
if lat[0] > lat[-1]:
    lat = lat[::-1]
    ano = ano[::-1, :]  # Adjust data dimensions accordingly

# Create grid (if lat/lon are 1D, need meshgrid)
lon_grid, lat_grid = np.meshgrid(lon, lat)

# Create figure and map projection
fig = plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

# Add map features
ax.add_feature(cfeature.LAND, edgecolor='black')
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAKES, alpha=0.5)
ax.add_feature(cfeature.RIVERS)

# Plot data (transpose ano to match lat/lon grid if needed)
contour = ax.contourf(lon_grid, lat_grid, ano,  # No transpose needed
                      60, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')

# Add colorbar
cbar = plt.colorbar(
    contour, ax=ax, orientation='horizontal', label='Anomaly Value',
    shrink=0.6, aspect=20
)
cbar.ax.set_xlabel(
    f'Min: {ano.min():.2f} | Max: {ano.max():.2f}',
    fontsize=10, ha='center'
)

# Add title and grid lines
plt.title(f'Global Temperature Anomaly Map for Month {int(month_value)}')
ax.gridlines(draw_labels=True, linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()